In [1]:
from evaluation_utils import *

In [2]:
def get_difficulty_first_chosen_by_user(user):
  user_to_features_data = get_all_features_data()
  features_data = user_to_features_data.get(user, None)
  if features_data == None:
    return 'nothing'
  if len(features_data['difficulty_items']) == 0:
    return 'nothing'
  return features_data['difficulty_items'][0]['difficulty']

In [16]:
def get_difficulty_chosen_by_user_at_idx(user, idx):
  user_to_features_data = get_all_features_data()
  features_data = user_to_features_data.get(user, None)
  if features_data == None:
    return 'nothing'
  return features_data['difficulty_items'][idx]['difficulty']


In [3]:


def get_initial_difficulty_v2(tensor, parameter_info_list):
  feature = tensor['feature']
  history_length = feature.size()[0]
  timestep = 0
  #if len(feature) < 2:
  #  return 'nothing'
  enabled_feature_list = get_enabled_features_list_from_parameter_info_list(parameter_info_list)
  get_feature_index = make_get_index(enabled_feature_list)
  difficulty_feature_idx = get_feature_index('initial_difficulty')
  for difficulty_idx,difficulty in enumerate(['nothing', 'easy', 'medium', 'hard']):
    if feature[timestep][0][difficulty_feature_idx + difficulty_idx].item() > 0:
      return difficulty
  return 'nothing'

def get_first_chosen_difficulty_v2(tensor, parameter_info_list):
  user = tensor['user']
  feature = tensor['feature']
  history_length = feature.size()[0]
  if history_length < 2:
    return 'nothing'
  return get_difficulty_first_chosen_by_user(user)

def always_predict_most_common_previous_v2(tensor, parameter_info_list):
  feature = tensor['feature']
  history_length = feature.size()[0]
  if len(feature) < 2:
    return 'nothing'
  enabled_feature_list = get_enabled_features_list_from_parameter_info_list(parameter_info_list)
  get_feature_index = make_get_index(enabled_feature_list)
  difficulty_feature_idx = get_feature_index('difficulty')
  difficulty_choices = Counter()
  for timestep in range(0, history_length - 1):
    for difficulty_idx,difficulty in enumerate(['nothing', 'easy', 'medium', 'hard']):
      if feature[timestep][0][difficulty_feature_idx + difficulty_idx].item() > 0:
        difficulty_choices[difficulty] += 1
  output = most_common_key_for_counter(difficulty_choices)
  if output == None:
    return 'nothing'
  return output
  #return 'nothing'

In [4]:
def evaluate_function_model_on_dataset_v2(parameter_info_list, func, dataset, prefix='dev_') -> Dict[str, Any]:
  confusion = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
  correct = 0
  total = 0
  for item in dataset:
    category_tensor = item['category']
    feature_tensor = item['feature']
    #if feature_tensor.size()[0] == 0:
    #    continue
    difficulty = tensor_to_difficulty(category_tensor)
    difficulty_idx = get_difficulty_idx(difficulty)
    #predicted_tensor = make_prediction(model, feature_tensor.cuda())
    #predicted_difficulty = prediction_to_difficulty(predicted_tensor)
    predicted_difficulty = func(item, parameter_info_list)
    predicted_difficulty_idx = get_difficulty_idx(predicted_difficulty)
    if predicted_difficulty_idx == difficulty_idx:
      correct += 1
    total += 1
    confusion[difficulty_idx][predicted_difficulty_idx] += 1
  return {
    prefix + 'correct': correct,
    prefix + 'total': total,
    prefix + 'confusion': confusion,
  }

In [17]:
def get_baseline_names():
  return [
    'initial_difficulty',
    'first_chosen',
    'always_nothing',
    'most_common_previous',
  ]

@jsonmemo1arg
def get_evaluation_results_for_named_baseline_v4(baseline_name):
  baseline_name_to_func = {
    'initial_difficulty': get_initial_difficulty_v2,
    'first_chosen': get_first_chosen_difficulty_v2,
    'always_nothing': always_predict_nothing,
    'most_common_previous': always_predict_most_common_previous_v2,
  }
  true = True
  false = False
  parameter_info_list : List[Dict[str, Any]] = [{"name": "dataset_name", "type": "dataset", "values": ["2019_04_01"], "value": "2019_04_01"}, {"name": "model_name", "type": "model", "values": ["selfattentionlstm"], "value": "selfattentionlstm"}, {"name": "criterion", "type": "model", "values": ["NLLLoss"], "value": "NLLLoss"}, {"name": "learning_rate", "type": "model", "values": [0.005, 0.05, 0.0005, 5e-05], "value": 5e-05}, {"name": "window_embed_size", "type": "model", "values": [64, 128, 256, 512], "value": 256}, {"name": "difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "time_of_day", "type": "feature", "values": [true, false], "value": true}, {"name": "day_of_week", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_productivity", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_category", "type": "feature", "values": [true, false], "value": true}, {"name": "initial_difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "languages", "type": "feature", "values": [true, false], "value": true}, {"name": "num_prior_entries", "type": "dataparam", "values": [10, 20, 30, 40], "value": 10}, {"name": "sample_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "sample_difficulty_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "disable_prior_visit_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "disable_difficulty_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "enable_current_difficulty", "type": "dataparam", "values": [false, true], "value": false}, {"name": "num_features", "type": "model", "values": [277], "value": 277}]
  train_data,dev_data,test_data = get_data_for_parameters(parameter_info_list)
  baseline_func = baseline_name_to_func[baseline_name]
  return evaluate_function_model_on_dataset_v2(parameter_info_list, baseline_func, dev_data)

In [18]:
true = True
false = False
parameter_info_list = [{"name": "dataset_name", "type": "dataset", "values": ["2019_04_11"], "value": "2019_04_11"}, {"name": "model_name", "type": "model", "values": ["selfattentionlstm"], "value": "selfattentionlstm"}, {"name": "criterion", "type": "model", "values": ["NLLLoss"], "value": "NLLLoss"}, {"name": "learning_rate", "type": "model", "values": [0.005, 0.05, 0.0005, 5e-05], "value": 5e-05}, {"name": "window_embed_size", "type": "model", "values": [64, 128, 256, 512], "value": 256}, {"name": "difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "time_of_day", "type": "feature", "values": [true, false], "value": true}, {"name": "day_of_week", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_productivity", "type": "feature", "values": [true, false], "value": true}, {"name": "domain_category", "type": "feature", "values": [true, false], "value": true}, {"name": "initial_difficulty", "type": "feature", "values": [true, false], "value": true}, {"name": "languages", "type": "feature", "values": [true, false], "value": true}, {"name": "num_prior_entries", "type": "dataparam", "values": [10, 20, 30, 40], "value": 10}, {"name": "sample_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "sample_difficulty_every_n_visits", "type": "dataparam", "values": [1], "value": 1}, {"name": "disable_prior_visit_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "disable_difficulty_history", "type": "dataparam", "values": [false, true], "value": false}, {"name": "enable_current_difficulty", "type": "dataparam", "values": [false, true], "value": false}, {"name": "num_features", "type": "model", "values": [277], "value": 277}]
set_parameter_in_parameter_info_list(parameter_info_list, 'sample_every_n_visits', 1)
train_data,dev_data,test_data = get_data_for_parameters(parameter_info_list)



running get_data_for_parameters
{'name': 'num_prior_entries', 'type': 'dataparam', 'values': [10, 20, 30, 40], 'value': 10}
{'name': 'sample_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'sample_difficulty_every_n_visits', 'type': 'dataparam', 'values': [1], 'value': 1}
{'name': 'disable_prior_visit_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'disable_difficulty_history', 'type': 'dataparam', 'values': [False, True], 'value': False}
{'name': 'enable_current_difficulty', 'type': 'dataparam', 'values': [False, True], 'value': False}
running get_all_features_data
all_features_data is
<class 'dict'>
all_data_tensors is
<class 'list'>
running split_into_train_dev_test


In [19]:
def get_nearest_visit_idx_for_sample_rate(visit_idx, sample_rate):
  num_visits_ago = visit_idx % sample_rate
  return visit_idx - num_visits_ago

In [28]:
@jsonmemo1arg
def get_evaluation_results_for_sample_every_n_visits_v3(sample_every_n_visits):
  true = True
  false = False
  def evaluation_func(tensor, parameter_info_list):
    user = tensor['user']
    visit_idx = tensor['visit_idx']
    if visit_idx < 1:
      return 'nothing'
    sampled_visit_idx = get_nearest_visit_idx_for_sample_rate(visit_idx - 1, sample_every_n_visits)
    return get_difficulty_chosen_by_user_at_idx(user, sampled_visit_idx)
  return evaluate_function_model_on_dataset_v2(parameter_info_list, evaluation_func, dev_data)
  

Created cache directory 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3


In [66]:
@jsonmemo1arg
def get_evaluation_results_for_sample_every_n_seconds_v2(sample_every_n_seconds):
  true = True
  false = False
  user_to_idx_to_difficulty = {}
  def evaluation_func(tensor, parameter_info_list):
    user = tensor['user']
    visit_idx = tensor['visit_idx']
    if user not in user_to_idx_to_difficulty:
      user_to_idx_to_difficulty[user] = get_visit_idx_to_predictions_for_timed_sampling_rate(user, sample_every_n_seconds)
    #if visit_idx < 1:
    #  return 'nothing'
    #sampled_visit_idx = get_nearest_visit_idx_for_sample_rate(visit_idx - 1, sample_every_n_visits)
    return user_to_idx_to_difficulty[user][visit_idx]
  return evaluate_function_model_on_dataset_v2(parameter_info_list, evaluation_func, dev_data)
  

Created cache directory 2019_04_11/get_evaluation_results_for_sample_every_n_seconds_v2


In [64]:
def get_visit_idx_to_predictions_for_timed_sampling_rate(user, sampling_rate_seconds):
  user_to_features_data = get_all_features_data()
  features_data = user_to_features_data.get(user, None)
  if features_data == None:
    return 'nothing'
  sampled_according_to_time = []
  last_sampled_time = None
  last_sampled_value = 'nothing'
  for idx,item in enumerate(features_data['difficulty_items']):
    difficulty = item['difficulty']
    arrow_time = item['arrow_time']
    sampled_according_to_time.append(last_sampled_value)
    if last_sampled_time != None:
      seconds_since_last_sample = (arrow_time - last_sampled_time).total_seconds()
      if seconds_since_last_sample >= sampling_rate_seconds:
        last_sampled_time = arrow_time
        last_sampled_value = difficulty
    else:
      last_sampled_time = arrow_time
      last_sampled_value = difficulty
  return sampled_according_to_time


In [65]:
def main():
  for baseline_name in ['initial_difficulty', 'first_chosen', 'always_nothing', 'most_common_previous']:
    print(baseline_name)
    print(get_evaluation_results_for_named_baseline_v4(baseline_name))
  
  sample_every_n_visits_options = [1,2,3,4,5,7,9,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,105,110,115,120,125,130]
  sample_every_n_visits_options.extend(list(range(1, 1000)))
  for sample_every_n_visits in sample_every_n_visits_options:
    print(sample_every_n_visits)
    print(get_evaluation_results_for_sample_every_n_visits_v3(sample_every_n_visits))


In [31]:
# noexport

main()

initial_difficulty
{'dev_correct': 2417, 'dev_total': 6674, 'dev_confusion': [[167, 1561, 552, 588], [285, 1385, 526, 157], [14, 297, 756, 60], [4, 27, 186, 109]]}
first_chosen
{'dev_correct': 3453, 'dev_total': 6674, 'dev_confusion': [[1136, 999, 434, 299], [287, 1473, 481, 112], [114, 246, 736, 31], [50, 25, 143, 108]]}
always_nothing
{'dev_correct': 2868, 'dev_total': 6674, 'dev_confusion': [[2868, 0, 0, 0], [2353, 0, 0, 0], [1127, 0, 0, 0], [326, 0, 0, 0]]}
most_common_previous
{'dev_correct': 5409, 'dev_total': 6674, 'dev_confusion': [[2459, 276, 62, 71], [275, 1904, 139, 35], [108, 117, 869, 33], [65, 27, 57, 177]]}
1
exception in jsonmemo1arg for file 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/1.json
[Errno 2] No such file or directory: '2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/1.json'
performing computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/1.json for arg 1
done with computation 2019_04_11/get_evaluation_res

done with computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/40.json
{'dev_correct': 4032, 'dev_total': 6674, 'dev_confusion': [[1602, 770, 242, 254], [304, 1532, 410, 107], [114, 206, 775, 32], [54, 23, 126, 123]]}
45
exception in jsonmemo1arg for file 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/45.json
[Errno 2] No such file or directory: '2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/45.json'
performing computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/45.json for arg 45
done with computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/45.json
{'dev_correct': 3977, 'dev_total': 6674, 'dev_confusion': [[1610, 766, 257, 235], [308, 1487, 452, 106], [115, 209, 772, 31], [67, 25, 126, 108]]}
50
exception in jsonmemo1arg for file 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/50.json
[Errno 2] No such file or directory: '2019_04_11/get_evaluation_results_for_sample_ever

done with computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/130.json
{'dev_correct': 3487, 'dev_total': 6674, 'dev_confusion': [[1170, 992, 407, 299], [287, 1473, 481, 112], [114, 246, 736, 31], [50, 25, 143, 108]]}
1
{'dev_correct': 5502, 'dev_total': 6674, 'dev_confusion': [[2550, 216, 42, 60], [296, 1906, 127, 24], [109, 112, 862, 44], [71, 21, 50, 184]]}
2
{'dev_correct': 5422, 'dev_total': 6674, 'dev_confusion': [[2503, 241, 55, 69], [294, 1896, 145, 18], [112, 121, 847, 47], [73, 21, 56, 176]]}
3
{'dev_correct': 5372, 'dev_total': 6674, 'dev_confusion': [[2481, 248, 62, 77], [292, 1861, 168, 32], [113, 107, 860, 47], [76, 13, 67, 170]]}
4
{'dev_correct': 5283, 'dev_total': 6674, 'dev_confusion': [[2438, 290, 64, 76], [309, 1837, 176, 31], [106, 137, 835, 49], [64, 24, 65, 173]]}
5
{'dev_correct': 5249, 'dev_total': 6674, 'dev_confusion': [[2377, 329, 79, 83], [313, 1862, 149, 29], [113, 138, 833, 43], [60, 29, 60, 177]]}
6
exception in jsonmemo1arg for fi

{'dev_correct': 4613, 'dev_total': 6674, 'dev_confusion': [[2040, 482, 166, 180], [330, 1643, 302, 78], [118, 164, 784, 61], [52, 22, 106, 146]]}
23
exception in jsonmemo1arg for file 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/23.json
[Errno 2] No such file or directory: '2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/23.json'
performing computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/23.json for arg 23
done with computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/23.json
{'dev_correct': 4584, 'dev_total': 6674, 'dev_confusion': [[2002, 519, 161, 186], [299, 1635, 336, 83], [113, 170, 802, 42], [53, 25, 103, 145]]}
24
exception in jsonmemo1arg for file 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/24.json
[Errno 2] No such file or directory: '2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/24.json'
performing computation 2019_04_11/get_evaluation_results_for_sample_ever

{'dev_correct': 3962, 'dev_total': 6674, 'dev_confusion': [[1604, 768, 259, 237], [309, 1482, 455, 107], [115, 213, 768, 31], [52, 40, 126, 108]]}
47
exception in jsonmemo1arg for file 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/47.json
[Errno 2] No such file or directory: '2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/47.json'
performing computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/47.json for arg 47
done with computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/47.json
{'dev_correct': 3907, 'dev_total': 6674, 'dev_confusion': [[1576, 764, 289, 239], [295, 1455, 495, 108], [114, 214, 768, 31], [52, 40, 126, 108]]}
48
exception in jsonmemo1arg for file 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/48.json
[Errno 2] No such file or directory: '2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/48.json'
performing computation 2019_04_11/get_evaluation_results_for_sample_ev

...

{'dev_correct': 3453, 'dev_total': 6674, 'dev_confusion': [[1136, 999, 434, 299], [287, 1473, 481, 112], [114, 246, 736, 31], [50, 25, 143, 108]]}
927
exception in jsonmemo1arg for file 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/927.json
[Errno 2] No such file or directory: '2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/927.json'
performing computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/927.json for arg 927
done with computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/927.json
{'dev_correct': 3453, 'dev_total': 6674, 'dev_confusion': [[1136, 999, 434, 299], [287, 1473, 481, 112], [114, 246, 736, 31], [50, 25, 143, 108]]}
928
exception in jsonmemo1arg for file 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/928.json
[Errno 2] No such file or directory: '2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/928.json'
performing computation 2019_04_11/get_evaluation_results_for_

{'dev_correct': 3453, 'dev_total': 6674, 'dev_confusion': [[1136, 999, 434, 299], [287, 1473, 481, 112], [114, 246, 736, 31], [50, 25, 143, 108]]}
945
exception in jsonmemo1arg for file 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/945.json
[Errno 2] No such file or directory: '2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/945.json'
performing computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/945.json for arg 945
done with computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/945.json
{'dev_correct': 3453, 'dev_total': 6674, 'dev_confusion': [[1136, 999, 434, 299], [287, 1473, 481, 112], [114, 246, 736, 31], [50, 25, 143, 108]]}
946
exception in jsonmemo1arg for file 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/946.json
[Errno 2] No such file or directory: '2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/946.json'
performing computation 2019_04_11/get_evaluation_results_for_

done with computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/961.json
{'dev_correct': 3453, 'dev_total': 6674, 'dev_confusion': [[1136, 999, 434, 299], [287, 1473, 481, 112], [114, 246, 736, 31], [50, 25, 143, 108]]}
962
exception in jsonmemo1arg for file 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/962.json
[Errno 2] No such file or directory: '2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/962.json'
performing computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/962.json for arg 962
done with computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/962.json
{'dev_correct': 3453, 'dev_total': 6674, 'dev_confusion': [[1136, 999, 434, 299], [287, 1473, 481, 112], [114, 246, 736, 31], [50, 25, 143, 108]]}
963
exception in jsonmemo1arg for file 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/963.json
[Errno 2] No such file or directory: '2019_04_11/get_evaluation_results_for_sa

{'dev_correct': 3453, 'dev_total': 6674, 'dev_confusion': [[1136, 999, 434, 299], [287, 1473, 481, 112], [114, 246, 736, 31], [50, 25, 143, 108]]}
977
exception in jsonmemo1arg for file 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/977.json
[Errno 2] No such file or directory: '2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/977.json'
performing computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/977.json for arg 977
done with computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/977.json
{'dev_correct': 3453, 'dev_total': 6674, 'dev_confusion': [[1136, 999, 434, 299], [287, 1473, 481, 112], [114, 246, 736, 31], [50, 25, 143, 108]]}
978
exception in jsonmemo1arg for file 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/978.json
[Errno 2] No such file or directory: '2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/978.json'
performing computation 2019_04_11/get_evaluation_results_for_

done with computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/991.json
{'dev_correct': 3453, 'dev_total': 6674, 'dev_confusion': [[1136, 999, 434, 299], [287, 1473, 481, 112], [114, 246, 736, 31], [50, 25, 143, 108]]}
992
exception in jsonmemo1arg for file 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/992.json
[Errno 2] No such file or directory: '2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/992.json'
performing computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/992.json for arg 992
done with computation 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/992.json
{'dev_correct': 3453, 'dev_total': 6674, 'dev_confusion': [[1136, 999, 434, 299], [287, 1473, 481, 112], [114, 246, 736, 31], [50, 25, 143, 108]]}
993
exception in jsonmemo1arg for file 2019_04_11/get_evaluation_results_for_sample_every_n_visits_v3/993.json
[Errno 2] No such file or directory: '2019_04_11/get_evaluation_results_for_sa

In [36]:
# noexport

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

baseline_name_and_correctness : List[Tuple[str, float]] = []
for baseline_name in get_baseline_names():
  results = get_evaluation_results_for_named_baseline_v4(baseline_name)
  correctness : float = float(results['dev_correct'] / results['dev_total'])
  baseline_name_and_correctness.append((baseline_name, correctness))

print(baseline_name_and_correctness)

[('initial_difficulty', 0.36215163320347615), ('first_chosen', 0.5173808810308661), ('always_nothing', 0.42972729997003295), ('most_common_previous', 0.8104584956547798)]


In [74]:
# scatterplot = []
# for sample_every_n_visits in range(1, 200):
#   results = get_evaluation_results_for_sample_every_n_visits_v3(sample_every_n_visits)
#   scatterplot.append([sample_every_n_visits, results['dev_correct'] / results['dev_total']])




# # Create a trace
# trace = go.Scatter(
#     x = [x[0] for x in scatterplot],
#     y = [x[1] for x in scatterplot],
#     mode = 'markers'
# )

# data = [trace]

# # Plot and embed in ipython notebook!
# iplot(data)


In [75]:
# scatterplot = []
# for n_seconds in range(1, 60*60*24*60, 60):
#   results = get_evaluation_results_for_sample_every_n_seconds_v2(n_seconds)
#   scatterplot.append([n_seconds / 3600, results['dev_correct'] / results['dev_total']])




# # Create a trace
# trace = go.Scatter(
#     x = [x[0] for x in scatterplot],
#     y = [x[1] for x in scatterplot],
#     mode = 'markers'
# )

# data = [trace]

# # Plot and embed in ipython notebook!
# iplot(data)
